testing jupyter notebook from vscode

In [81]:
# imports
import pandas as pd
import requests
import altair as alt

In [82]:
alt.renderers.enable('altair_viewer')

RendererRegistry.enable('altair_viewer')

In [83]:
# set up alphavantage api stuff
# symbol = input()
api_key = 'LAJBI8EN52GC18OY'
base_url = 'https://www.alphavantage.co/query?'
params = {'function': 'TIME_SERIES_DAILY_ADJUSTED',
          'symbol': 'CYDY',
          'apikey': api_key}

In [84]:
# get response from api and format it a bit and put it into a dict
# this helps with the mulit level nested json by 'stripping' metadata header info
response = requests.get(base_url, params=params)
response_dict = response.json()
_, header = response.json()

In [85]:
# create a pandas dataframe from response_dict
df = pd.DataFrame.from_dict(response_dict[header], orient='index')
# strip the cols names down to bare bones header
df_cols = [i.split(' ')[1] for i in df.columns]
df.columns = df_cols

In [86]:
df['date'] = df.index

In [87]:
df

,open,high,low,close,adjusted,volume,dividend,split,date
2021-03-09,2.7,2.7,2.17,2.35,2.35,22115724,0.0000,1.0,2021-03-09
2021-03-08,2.9,3.28,2.7,2.91,2.91,21383775,0.0000,1.0,2021-03-08
2021-03-05,4.2,4.75,4.02,4.05,4.05,4604187,0.0000,1.0,2021-03-05
2021-03-04,4.35,4.35,4.02,4.06,4.06,3319689,0.0000,1.0,2021-03-04
2021-03-03,4.59,4.62,4.3,4.32,4.32,2219446,0.0000,1.0,2021-03-03
...,...,...,...,...,...,...,...,...,...
2020-10-20,2.99,3.09,2.66,2.7301,2.7301,6170210,0.0000,1.0,2020-10-20
2020-10-19,2.82,2.93,2.81,2.82,2.82,1863826,0.0000,1.0,2020-10-19
2020-10-16,2.85,2.89,2.76,2.81,2.81,4500498,0.0000,1.0,2020-10-16
2020-10-15,2.94,3.06,2.89,2.98,2.98,2922052,0.0000,1.0,2020-10-15


In [88]:
#this saves a new df with a new index 01234 etc not the date string.
df1 = df.reset_index(drop=True)

In [90]:
#this works now with all cols as 'objects'. no need to convert 'date' col to datetime in pandas
chart = alt.Chart(df1).mark_circle(size=50).encode(
    x='date:T',
    y='open:Q',
    
    tooltip=['open','close','high','low']
).interactive()
chart.show()

Displaying chart at http://localhost:23566/


In [94]:
# notice the new index field and 'date' field
df1

,open,high,low,close,adjusted,volume,dividend,split,date
0,2.7,2.7,2.17,2.35,2.35,22115724,0.0000,1.0,2021-03-09
1,2.9,3.28,2.7,2.91,2.91,21383775,0.0000,1.0,2021-03-08
2,4.2,4.75,4.02,4.05,4.05,4604187,0.0000,1.0,2021-03-05
3,4.35,4.35,4.02,4.06,4.06,3319689,0.0000,1.0,2021-03-04
4,4.59,4.62,4.3,4.32,4.32,2219446,0.0000,1.0,2021-03-03
...,...,...,...,...,...,...,...,...,...
95,2.99,3.09,2.66,2.7301,2.7301,6170210,0.0000,1.0,2020-10-20
96,2.82,2.93,2.81,2.82,2.82,1863826,0.0000,1.0,2020-10-19
97,2.85,2.89,2.76,2.81,2.81,4500498,0.0000,1.0,2020-10-16
98,2.94,3.06,2.89,2.98,2.98,2922052,0.0000,1.0,2020-10-15


In [95]:
df_difs = df1[['date','open','close']]

In [96]:
df_difs

,date,open,close
0,2021-03-09,2.7,2.35
1,2021-03-08,2.9,2.91
2,2021-03-05,4.2,4.05
3,2021-03-04,4.35,4.06
4,2021-03-03,4.59,4.32
...,...,...,...
95,2020-10-20,2.99,2.7301
96,2020-10-19,2.82,2.82
97,2020-10-16,2.85,2.81
98,2020-10-15,2.94,2.98


In [101]:
# this is computing the difference between 'open' and 'close' and putting result in the 'close' col after converting from object to numeric(float64). # otherwise math won't work
dfn = df_difs[['open', 'close']].apply(pd.to_numeric).diff(axis=1)

In [102]:
dfn

,open,close
0,NaN,-0.3500
1,NaN,0.0100
2,NaN,-0.1500
3,NaN,-0.2900
4,NaN,-0.2700
...,...,...
95,NaN,-0.2599
96,NaN,0.0000
97,NaN,-0.0400
98,NaN,0.0400
